In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# 데이터 불러오기

In [ ]:
df_n = pd.read_csv('./data/개별기업뉴스_자연어처리/AJ네트웍스.csv',index_col=0)
df_n['뉴스'] = df_n['뉴스'].apply(lambda x : eval(x))
df_n.index = pd.to_datetime(df_n.index)

# 키워드 분석

## 특정구간

In [ ]:
# 특정구간 단어 빈도수 + 시각화
kw_dict = dict()

# 2016-01-01 ~ 2016-02-01 가정
bgn_date = '2016-01-01'
end_date = '2016-02-01'

token_lst = df_n['뉴스'].loc[bgn_date:end_date]

for tokens in token_lst:

    for word in tokens:
        if not word in kw_dict.keys():
            kw_dict[word] = 1
        else:
            kw_dict[word] += 1

kw_dict = dict(sorted(kw_dict.items(), key = lambda x: x[1]))

plt.figure(figsize=(15,10))
keyword_list = list(kw_dict.keys())
keyword_count = list(kw_dict.values())
plt.barh(keyword_list,keyword_count)
plt.show()

## 전체구간

In [ ]:
kw_dict1 = dict()

# 2016-01-01 ~ 2022-03-31
bgn_date = '2016-01-01'
end_date = '2022-03-31'

token_lst = df_n['뉴스'].loc[bgn_date:end_date]

for tokens in token_lst:

    for word in tokens:
        if not word in kw_dict1.keys():
            kw_dict1[word] = 1
        else:
            kw_dict1[word] += 1

kw_dict1 = dict(sorted(kw_dict1.items(), key = lambda x: x[1]))

kw_dict1

In [ ]:
# 1단어 제거
for key,value in list(kw_dict2.items()):
    if value ==1 :
        del kw_dict2[key]

In [ ]:
# 본인 회사이름 제거
for key,value in list(kw_dict2.items()):
    if 'AJ네트웍스' in key :
        del kw_dict2[key]
kw_dict2

In [ ]:
# 뽑은 불용어 제거
df_nsw = pd.read_csv('./data/뉴스불용어.csv',index_col=1)
df_nsw.reset_index(drop=True, inplace=True)

for key,value in list(kw_dict2.items()):
    for word in df_nsw['불용어']:
        if word in key :
            try:
                del kw_dict2[key]
            except:
                pass
print(len(kw_dict2))
kw_dict2

In [ ]:
df_keyword = pd.DataFrame()
keyword_lst = list(kw_dict1.keys())

for keyword in keyword_lst:
    # 출현 날짜
    rp_date = []
    # 날짜당 출현 횟수
    cnt_list = []

    df_rp = df_n['뉴스'].loc[bgn_date:end_date]

    for i in df_rp.index:
        # 날짜
        rp_date.append(i)
        # 횟수
        cnt = 0
        tokens = df_n['뉴스'].loc[i]
        for token in tokens:
            if keyword in token:
                cnt +=1

        cnt_list.append(cnt)

    df_rp = pd.DataFrame({'날짜':rp_date, '출현횟수':cnt_list})
    df_rp = df_rp.set_index('날짜')
    df_keyword = pd.concat([df_keyword,df_rp],axis=1)
    
df_keyword.columns = keyword_lst
df_keyword